In [37]:
import json
from pprint import pprint
from matplotlib import pyplot as plt
import cv2
import copy
import operator
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
%matplotlib qt

In [2]:
with open('DS3.json', 'r') as fp:
    ds3_data = json.load(fp)

with open('DS2.json', 'r') as fp:
    ds2_data = json.load(fp)
    
with open('DS4.json', 'r') as fp:
    ds4_data = json.load(fp)

In [3]:
def section_separator(data, start, end):
    
    exercise_one = {}
    exercise_two = {}
    
                
    for i in range(start, end + 1):
        
        for j in data[str(i)]:
            if j == '0':
                exercise_one[i] = data[str(i)][j]
            else:
                exercise_two[i] = data[str(i)][j]
        
#         i += 1

    return exercise_one, exercise_two

In [4]:
def plotSkeleton(data):
    
    fig = plt.subplot()
    
    fig.set_xlim(0, 1920)
    fig.set_ylim(0, 1080)
    first = True
    
#     fig.set_ylim(fig.get_ylim()[::-1])

    for frame in data:
    
        if not first:
            fig.clear()
        else:
            first = False
            
        fig.scatter(0,1080)
        fig.scatter(1920, 1080)
        fig.scatter(0,0)
            
        if len(data[frame]) != 0:
            for body_part in data[frame]:
                fig.scatter(data[frame][body_part][0], 1080 - data[frame][body_part][1])
        plt.pause(0.001)
        
    plt.show()

In [5]:
def section_player(file, data, start, end, output = ""):
    fvs = cv2.VideoCapture(file)
    
#     video = cv2.VideoWriter(output,cv2.VideoWriter_fourcc(*"MJPG"), 30, (1920,1080),True)

    
    if start == 0:
        success, frame = fvs.read()
    else:
        for k in range(start):
            success, frame = fvs.read()
                
    for i in range(start, end):
        people = len(data[str(i)])
        
        for j in data[str(i)]:
            tmp_points = copy.deepcopy(data[str(i)][j])
            x_points = copy.deepcopy(data[str(i)][j])
            y_points = {}
            total_del = 0
            
            for point in tmp_points:
                if x_points[point][0] == 0:
                    del x_points[point]
                    total_del += 1
                if tmp_points[point][1] != 0:
                    y_points[point] = tmp_points[point][1]
                    
            
            minY = min(y_points.items(), key = operator.itemgetter(1))[1]
            maxY = max(y_points.items(), key = operator.itemgetter(1))[1]


            maxX, GY = max(x_points.items(), key = operator.itemgetter(1))[1]
            minX, GY = min(x_points.items(), key = operator.itemgetter(1))[1]

            minY = int(minY)
            maxY = int(maxY)
            minX = int(minX)
            maxX = int(maxX)
            
            cv2.rectangle(frame, (minX, minY), (maxX, maxY), (0, 255, 0), 3)
            cv2.putText(frame, str(j), (int(minX + (maxX - minX) / 2), int(minY + (maxY - minY) / 2)), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 255, 255), 10, cv2.LINE_AA)

        cv2.imshow('Video', frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
            
#         video.write(frame)

        success, frame = fvs.read()
        
#         i += 1

#     video.release()
    cv2.destroyAllWindows()


In [6]:
def organiseAllPoints(data):
    new_data = {}
    
    # Initialisation
    for keys in data[list(data.keys())[0]]:
        
        new_data[keys] = []
       
    for keys in data:
        for Lkeys in data[keys]:
            coordinate = new_data[Lkeys]
            
            points = data[keys][Lkeys]
            
            if points[0] != 0 and points[1] != 0:
                coordinate.append(points)
    
            
            new_data[Lkeys] = coordinate            
    
    return new_data

In [59]:
def getMeanAndVariace(data):
    
    allPoints = organiseAllPoints(data)
    
    new_data = {}
    
    for keys in allPoints:
        ini = {}
        ini['x'] = []
        ini['y'] = []
        
        new_data[keys] = ini
        
    fit_points = []
    first = True
    
    for keys in allPoints:
        if first:
            fit_points = allPoints[keys]
            first = False
        else:
            fit_points + allPoints[keys]
            
    scaler = StandardScaler().fit(fit_points)
    
    for keys in allPoints:
        points = scaler.transform(allPoints[keys])
        
        x = points[:, 0]
        y = points[:, 1]
        x_mean = np.mean(x)
        x_std = np.std(x)
        y_mean = np.mean(y)
        y_std = np.std(y)
        
        X = [x_mean, x_std]
        Y = [y_mean, y_std]
        
        new_data[keys]['x'] = X
        new_data[keys]['y'] = Y
        
    return new_data

In [8]:
# section_player("DS3.MOV", ds4_data, 3580, 4370)
# 210, 980
# 1430, 2210
#2550, 3330
#3580, 4370

In [9]:
squat1, arm1 = section_separator(ds2_data, 135, 800)
arm2, squat2 = section_separator(ds2_data, 1230, 1910)
squat3, arm3 = section_separator(ds2_data, 2700, 3350)
arm4, squat4 = section_separator(ds2_data, 3600, 4280)

In [10]:
squat5, arm5 = section_separator(ds3_data, 210, 930)
arm6, squat6 = section_separator(ds3_data, 1250, 1990)
squat7, arm7 = section_separator(ds3_data, 2330, 2980)
arm8, squat8 = section_separator(ds3_data, 3725, 4350)
squat9, arm9 = section_separator(ds3_data, 4585, 5325)

In [11]:
squat10, starJump1 = section_separator(ds4_data, 210, 980)
starJump2, squat11 = section_separator(ds4_data, 1430, 2210)
squat12, starJump2 = section_separator(ds4_data, 2550, 3330)
starJump3, squat13 = section_separator(ds4_data, 3580, 4370)

In [12]:
x = np.linspace(210, 980, 980 - 210)

In [65]:
pprint(getMeanAndVariace(arm8))

{'LAnkle': {'x': [2.4115965959208427, 0.8177722257438697],
            'y': [30.04896801575727, 0.1547219778356454]},
 'LBigToe': {'x': [1.721198565310174, 0.9038831948731916],
             'y': [31.268541136164316, 0.03931266319666221]},
 'LEar': {'x': [1.0906235067744643, 0.2510126193220549],
          'y': [-0.43485224800231054, 0.2590177003317841]},
 'LElbow': {'x': [1.2141717239204215, 1.0214006108472982],
            'y': [9.053404381134417, 0.6208860988344125]},
 'LEye': {'x': [0.28890251817920337, 0.995774008526443],
          'y': [-0.5578010584328398, 0.8853647877134785]},
 'LHeel': {'x': [2.704503672057455, 0.7806876660333277],
           'y': [30.801698544562285, 0.22456154546425108]},
 'LHip': {'x': [1.1028284815587657, 0.8572068132852694],
          'y': [14.674875639175815, 0.3341006807603924]},
 'LKnee': {'x': [1.7027569313781992, 0.9480223773506495],
           'y': [22.796569555275813, 0.23162304263501285]},
 'LShoulder': {'x': [1.6689736608197863, 0.9359332799349007]